In [1]:
import numpy as np
import pandas as pd
import os
import re
import math
import random
import pickle
import tensorflow as tf
import matplotlib.dates as mdates
from datetime import time
xformatter = mdates.DateFormatter('%H:%M')  # for time axis plots
import datetime
from dateutil.parser import parse
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib inline
from pandas.tseries.frequencies import to_offset
from pickle import load,dump
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,f_regression
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM,GRU,Bidirectional
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.backend import clear_session
import warnings
warnings.filterwarnings('ignore')

In [2]:
FORMAT_TIME='%Y-%m-%d %H:%M:%S'
START_TIME_DAY = time(5,0,0)
END_TIME_DAY = time(18,30,0)
TIME_1_STEP = 15
step_lag_1_day = 24*60//TIME_1_STEP
steps_2hours = 60*2//TIME_1_STEP

In [3]:
# Nhap du lieu train data
def import_train_data(path_file_train):
    df = pd.read_csv(path_file_train)
    df['TimeStamp'] = pd.to_datetime(df['TimeStamp']
                                      )
    df = df.set_index('TimeStamp')
    return df

In [4]:
def resample_df(df, resample_time, time_col='TimeStamp'):
    """
    resample_time: `minute`
    """
    resample_df = df.copy()
    if resample_time >= 30:
        resample_df = resample_df.set_index(
            resample_df[time_col] - to_offset(str(resample_time//2)+"min"))
    resample_df = resample_df.resample(str(resample_time)+'min', label='right').mean()
    return resample_df

In [ ]:
# Split train, val data theo ti le 8/2
def train_valid_split(df, split_ratio=[0.8, 0.2]):
    train_ratio, valid_ratio = split_ratio
    assert train_ratio + valid_ratio  == 1.0
    n_df = len(df)
    # Train / Validation  Split
    train_split = int(n_df * train_ratio)
    valid_split = int(n_df * (train_ratio + valid_ratio))

    train = df[:train_split]
    val = df[train_split:valid_split]

    print(f'Train set: {len(train)} ')
    print(f'Validation set: {len(val)} ')

    return train, val 

In [5]:
def load_scale(df, path_scale):
    scale_df = pd.DataFrame(index=df.index)
    for col in df.columns:
        scaler = pickle.load(open(path_scale + col+'.pkl','rb'))
        scale_df[col] = scaler.transform(df[col].values.reshape(-1,1))[:,0]
    return scale_df

In [ ]:
def num_step_1hour(df):
    """
    Get number step of 1 hours
    """
    step_hours = None
    if type(df.index) == pd.core.indexes.datetimes.DatetimeIndex:
        time_1step = int((df.index[1] - df.index[0]) /
                         np.timedelta64(1, 'm'))  # minute
        step_hours = 60 // time_1step
    return step_hours

In [ ]:
def make_data_supervised(dt, num_pre_around=5, num_day_pre=4):
    step_lag_1_day = num_step_1hour(dt)*24
    dt_lag = pd.DataFrame()
    for col in dt.columns:
        for day_pre in range(num_pre_around+1):
            if day_pre == 0:
                dt_lag[col+'(t)'] = dt[col]
            else:
                dt_lag[col+'(t-'+str(day_pre)+')'] = dt[col].shift(day_pre)
        for day_pre in range(1, num_day_pre):
            step_lag = step_lag_1_day*day_pre
            for lag in range(num_pre_around+1):
                dt_lag[col+'(t-'+str(step_lag+lag) +
                       ')'] = dt[col].shift(step_lag+lag)
                dt_lag[col+'(t-'+str(step_lag-lag) +
                       ')'] = dt[col].shift(step_lag-lag)
    dt_lag = dt_lag.dropna()
    dt_lag['hour'] = dt_lag.index.hour
    dt_lag['day'] = dt_lag.index.day
    dt_lag['day_of_week'] = dt_lag.index.dayofweek
    dt_lag['month'] = dt_lag.index.month
    dt_lag['day_of_year'] = dt_lag.index.dayofyear
    return dt_lag

In [ ]:
def load_scale(df, path_scale):
    scale_df = pd.DataFrame(index=df.index)
    for col in df.columns:
        scaler = pickle.load(open(path_scale + col+'.pkl','rb'))
        scale_df[col] = scaler.transform(df[col].values.reshape(-1,1))[:,0]
    return scale_df

In [ ]:
def sort_df_by_column_list(df, column_list):
    return df.reindex(columns=column_list)

In [ ]:
def recal_best_feature(path_scale):
    best_feature = pickle.load(open(path_scale + 'best_feature.pkl','rb'))
    best_feature.insert(0, 'TotW(t)')
    return best_feature

In [ ]:
def import_new_data_training(name_file,time,path_scale):
    power_plant = import_train_data(name_file)
    data_resample = resample_df(power_plant,resample_time = time, time_col = 'TimeStamp')
    train_solar, val_solar = train_valid_split(data_resample,split_ratio=[0.8, 0.2])
    train_scale = load_scale(train_solar, path_scale)
    val_scale = load_scale(val_solar, path_scale)
    
    train_scaler_lag = make_data_supervised(train_scale)
    val_scaler_lag = make_data_supervised(val_scale)
    
    best_feature = recal_best_feature(path_scale)
    #Making supervised for train and val data
    train_solar = sort_df_by_column_list(train_scaler_lag, best_feature)
    val_solar = sort_df_by_column_list(val_scaler_lag, best_feature)
    train_X, train_y= train_solar.values[:, 1:],train_solar.values[:, :1]
    train_X = train_X.reshape(train_X.shape[0], 1, train_X.shape[1])
    val_X, val_y= val_solar.values[:, 1:],val_solar.values[:, :1]
    val_X = val_X.reshape(val_X.shape[0], 1, val_X.shape[1])
    
    return train_X,train_y,val_X,val_y
    

In [ ]:
train_X,train_y,val_X,val_y = import_new_data_training('./LN2_training_10.csv',15,'./LN2/part1/time1/')

In [ ]:
def training_new_data(path_model):
    model = load_model(path_model)
    for layer in model.layers:
        layer.trainable = True # False nếu không thay đổi trọng số và ngược lại
    
    model.compile(loss = 'mae', optimizer = tf.keras.optimizers.Adam(
                        lr=0.001,
                        beta_1=0.9,
                        beta_2=0.999,), 
                        metrics=['accuracy'])
    weights = model.get_weights()
    
    history = model.fit(train_X, train_y, validation_data=(val_X, val_y), callbacks = EarlyStopping(
                        monitor = 'val_loss',
                        patience = 15,
                        restore_best_weights=True),
                        epochs=80, batch_size=50, verbose=0,shuffle=False)      
    new_weight = model.get_weights()
#     model.save('./LN2/part1/time1/4H_change_weight_10.h5')
    return model, weights, history, new_weight
    

In [ ]:
new_model, weights, history, new_weight = training_new_data('./LN2/part1/time1/4H_change_weight_9.h5')

In [ ]:
weights

In [ ]:
new_model.weights

In [ ]:
for layer in new_model.layers:
    print('Layer: {} ; Trainable: {}'.format(layer, layer.trainable))


In [ ]:
new_model.trainable_weights #Trọng số có thể đào tạo

In [ ]:
plt.ylim(0, 0.3)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
history.history['loss'][-1]

In [ ]:
history.history['val_loss'][-1]

## Testing 11,12 

In [6]:
def cal_ape(true, pred, P_dm=187.158):
    ape = []
    for i in range(len(true)):
        ape.append(abs((true[i]-pred[i])/P_dm)*100)
    return ape

In [7]:
def find_number_in_str(s):
    '''s chỉ chứa nhiều nhất 1 numberic'''
    s_re = re.findall(r'\b\d+\b', s)
    if len(s_re)==0:
        return 0
    else:
        return int(s_re[0])

In [8]:
def find_lag_max(feature_list):
    lag_feature = []
    step_lag_max = 0
    for col in feature_list:
        lag = re.findall(r'\b\d+\b', col)
        if len(lag)==0:
            lag_feature=0
        else:
            lag_feature=int(re.findall(r'\b\d+\b', col)[0])
        step_lag_max = max(step_lag_max,lag_feature)
    return step_lag_max

In [9]:
def make_feature_lag_dict(feature_list):
    feature_lag = {}
    for col in feature_list:
        lag = find_number_in_str(col)
        feature = col.split('(')[0]
        feature_lag[col]=[feature, lag]
    return feature_lag

In [10]:
def create_features(df):
    """
    Creates time series features from datetime index
    """
    df['time'] = df.index.time
    df['hour'] = df.index.hour
    df['day'] = df.index.day
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    df['day_of_year'] = df.index.dayofyear
    return df

In [11]:
#Nhập test
file_test_path = './'
name_file = 'test_LN2_2.csv'
test_solar = pd.read_csv(file_test_path + name_file)

In [12]:
def recursiver_ahead(true_dt, first_time_pred, totw_scale,path_model, path_feature, p_dm=187.158, len_pred=16):
    if (first_time_pred.hour,first_time_pred.minute)==(18,30):
        return(None)
    
    pre_max_lag_list = true_dt[true_dt.index<=first_time_pred]['TotW']# <== list
    if (len(true_dt)-len(pre_max_lag_list))<len_pred:
        return(None)

    true, pred = [], []
    ind_list = []
    best_feature = pickle.load(open(path_feature + 'best_feature.pkl','rb')) #best_feature
    model = load_model(path_model) #model
    len_step_skip =len(pre_max_lag_list)
    feature_lag = make_feature_lag_dict(best_feature)
    for i in range(len_pred):    
        # get True and time
        t = len_step_skip + i-1
        time_true = true_dt.index[t].to_pydatetime()
        totW_true = true_dt['TotW'][t]
        true.append(totW_true)
        ind_list.append(true_dt.index[t])
        
        time_true = true_dt.index[t].to_pydatetime()
        X_values = {}
         
        for feature, value  in feature_lag.items():
            name, lag = value
            if 'TotW' in name:
                X_values[feature] = pre_max_lag_list[-lag]   
            else:
                X_values[feature] = true_dt[name][t-lag]
       # lấy ra các giá trị feature theo thứ tự
        list_X = []
        for f in best_feature:
            list_X.append(X_values[f])
       # Forecast
        is_day_time = (time_true.time() <= time(18, 30)) & (time_true.time() >= time(5, 0))
        if is_day_time:
            X = np.array([list_X[:len(best_feature)]]).reshape(1, 1, len(best_feature))
            pred_X = model.predict(X)[0, 0]
            pred_X = max(pred_X,0)  # loại các giá trị âm
        else:
            pred_X = 0
        pred.append(pred_X)
        
        # update các giá trị totW(t-lag)
        pre_max_lag_list = pre_max_lag_list.append(pd.Series(pred_X,index=[true_dt.index[t]]))
        pre_max_lag_list = pre_max_lag_list.iloc[1:]
        
    scaler_totW = pickle.load(open(totw_scale,'rb'))
    result_recur = pd.DataFrame()
    result_recur['Power_reality'] = scaler_totW.inverse_transform(np.array(true).reshape(-1,1))[:,0]
    result_recur['Power_forecast'] = scaler_totW.inverse_transform(np.array(pred).reshape(-1,1))[:,0] 
    result_recur['TimeStamp'] = ind_list
    result_recur.loc[result_recur.Power_forecast > p_dm, "Power_forecast"] = p_dm
    result_recur = result_recur.set_index('TimeStamp')
    return(result_recur)

In [13]:
def import_test(df,path_scale):
    df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
    df = df.set_index('TimeStamp')
    df = resample_df(df, resample_time = 15, time_col='TimeStamp')
    test_recursive = load_scale(df,path_scale)
    test_recursive = create_features(test_recursive)
    return test_recursive

In [14]:
test_recursive = import_test(test_solar,'./LN2/add_layer_3/')

In [15]:
ind_30min_day_time = test_recursive.resample('30min').sum().between_time(START_TIME_DAY, END_TIME_DAY).index[:-1]
test_recursive['time'] = test_recursive.index.time
test_recursive['hour'] = test_recursive.index.hour
test_recursive['day'] = test_recursive.index.day
test_recursive['day_of_week'] = test_recursive.index.dayofweek

test_recursive['month'] = test_recursive.index.month
test_recursive['day_of_year'] = test_recursive.index.dayofyear

In [ ]:
best_feature = pickle.load(open('./LN2/add_layer_3/' + 'best_feature.pkl','rb'))
lag_max_0 = find_lag_max(best_feature)
lag_max = max(lag_max_0, steps_2hours) + 1
i = lag_max
i_time= test_recursive.index[i]
result_dt = {}
lag_skip = math.ceil(lag_max/(96/28))
for i in range(len(ind_30min_day_time)-lag_skip-1):
    time_lag_max = ind_30min_day_time[i]
    i_time = ind_30min_day_time[lag_skip + i]
    print('\n',i_time)
    result_ind = recursiver_ahead(test_recursive[time_lag_max:],i_time,'./LN2/add_layer_3/TotW.pkl','./LN2/add_layer_3/4H.h5','./LN2/add_layer_3/')
    if result_ind is None:
        continue
    result_ind['APE'] = cal_ape(result_ind['Power_reality'],
                                result_ind['Power_forecast'],
                                P_dm=187.158)
    result_dt[i_time] = result_ind
    result_ind.to_csv('./test_LN2/transfer_test/add_layer/time_3/'+i_time.strftime('%Y-%m-%d %H %M %S')+'.csv')

    print(result_ind)
    print('---')
    result_ind = result_ind.reset_index()


 2022-11-03 06:00:00
                     Power_reality  Power_forecast        APE
TimeStamp                                                    
2022-11-03 06:00:00       0.000000        0.000000   0.000000
2022-11-03 06:15:00       0.000000        0.126108   0.067381
2022-11-03 06:30:00       0.870263        3.722780   1.524123
2022-11-03 06:45:00       4.479144       10.206856   3.060362
2022-11-03 07:00:00       7.426971       18.176078   5.743333
2022-11-03 07:15:00       9.033330       27.087189   9.646319
2022-11-03 07:30:00      10.847688       36.552017  13.734026
2022-11-03 07:45:00      16.230990       46.525837  16.186776
2022-11-03 08:00:00      26.341590       56.049796  15.873330
2022-11-03 08:15:00      30.864820       66.257972  18.910841
2022-11-03 08:30:00      34.230417       76.251304  22.452092
2022-11-03 08:45:00      36.872287       85.359497  25.907100
2022-11-03 09:00:00      55.498707       93.421768  20.262592
2022-11-03 09:15:00      68.933753      100.6971

                     Power_reality  Power_forecast        APE
TimeStamp                                                    
2022-11-03 10:00:00     106.971967      115.299416   4.449422
2022-11-03 10:15:00     133.993500      120.704758   7.100280
2022-11-03 10:30:00     131.348333      124.474121   3.672946
2022-11-03 10:45:00      87.928533      127.737518  21.270256
2022-11-03 11:00:00     113.268210      130.624725   9.273723
2022-11-03 11:15:00     111.214327      132.363297  11.300062
2022-11-03 11:30:00      35.456813      129.559357  50.279733
2022-11-03 11:45:00      10.844876      125.481583  61.251299
2022-11-03 12:00:00      12.883154      124.033813  59.388677
2022-11-03 12:15:00      27.190723      127.768677  53.739596
2022-11-03 12:30:00      37.566170      126.791512  47.673806
2022-11-03 12:45:00      28.761180      128.828491  53.466756
2022-11-03 13:00:00      27.526870      129.460541  54.463967
2022-11-03 13:15:00      26.996893      126.405373  53.114737
2022-11-

                     Power_reality  Power_forecast        APE
TimeStamp                                                    
2022-11-03 14:00:00      23.433930       37.095913   7.299706
2022-11-03 14:15:00      30.831597       37.999874   3.830067
2022-11-03 14:30:00      28.376197       43.625114   8.147617
2022-11-03 14:45:00      24.126850       51.138454  14.432514
2022-11-03 15:00:00      22.491083       57.586781  18.751909
2022-11-03 15:15:00      17.960000       58.854767  21.850397
2022-11-03 15:30:00      13.877513       56.581276  22.816958
2022-11-03 15:45:00      12.730817       52.848793  21.435352
2022-11-03 16:00:00      10.870383       46.707230  19.147910
2022-11-03 16:15:00       9.056037       38.960232  15.978048
2022-11-03 16:30:00      11.302533       30.675812  10.351296
2022-11-03 16:45:00       8.632788       22.649832   7.489417
2022-11-03 17:00:00       6.140884       15.439925   4.968551
2022-11-03 17:15:00       3.964714        8.985486   2.682638
2022-11-

                     Power_reality  Power_forecast  APE
TimeStamp                                              
2022-11-03 18:00:00            0.0             0.0  0.0
2022-11-03 18:15:00            0.0             0.0  0.0
2022-11-03 18:30:00            0.0             0.0  0.0
2022-11-03 18:45:00            0.0             0.0  0.0
2022-11-03 19:00:00            0.0             0.0  0.0
2022-11-03 19:15:00            0.0             0.0  0.0
2022-11-03 19:30:00            0.0             0.0  0.0
2022-11-03 19:45:00            0.0             0.0  0.0
2022-11-03 20:00:00            0.0             0.0  0.0
2022-11-03 20:15:00            0.0             0.0  0.0
2022-11-03 20:30:00            0.0             0.0  0.0
2022-11-03 20:45:00            0.0             0.0  0.0
2022-11-03 21:00:00            0.0             0.0  0.0
2022-11-03 21:15:00            0.0             0.0  0.0
2022-11-03 21:30:00            0.0             0.0  0.0
2022-11-03 21:45:00            0.0             0

                     Power_reality  Power_forecast        APE
TimeStamp                                                    
2022-11-04 08:30:00      21.805273       34.334675   6.694558
2022-11-04 08:45:00      19.344433       45.616142  14.037182
2022-11-04 09:00:00      21.145500       59.222694  20.344946
2022-11-04 09:15:00      43.337307       71.514214  15.055144
2022-11-04 09:30:00      97.838887       82.151649   8.381815
2022-11-04 09:45:00      92.261393       93.285660   0.547274
2022-11-04 10:00:00      71.680130      102.389381  16.408196
2022-11-04 10:15:00      59.052323      108.503906  26.422372
2022-11-04 10:30:00      66.589253      113.330971  24.974469
2022-11-04 10:45:00      78.168813      116.116089  20.275530
2022-11-04 11:00:00      92.394190      110.345482   9.591517
2022-11-04 11:15:00     100.301733      100.566299   0.141360
2022-11-04 11:30:00     106.763100       83.980064  12.173156
2022-11-04 11:45:00      85.154987       72.417549   6.805714
2022-11-

...

                     Power_reality  Power_forecast       APE
TimeStamp                                                   
2022-12-21 06:00:00       0.000000        0.000000  0.000000
2022-12-21 06:15:00       0.000000        0.000000  0.000000
2022-12-21 06:30:00       0.000000        0.059457  0.031768
2022-12-21 06:45:00       0.817201        1.729439  0.487416
2022-12-21 07:00:00       5.419744        8.615360  1.707443
2022-12-21 07:15:00      11.147798       18.084784  3.706486
2022-12-21 07:30:00      28.160833       28.027788  0.071087
2022-12-21 07:45:00      39.535210       39.719958  0.098712
2022-12-21 08:00:00      52.226177       50.407329  0.971825
2022-12-21 08:15:00      56.011423       60.901517  2.612816
2022-12-21 08:30:00      60.109840       71.243653  5.948884
2022-12-21 08:45:00      70.566523       80.912598  5.527989
2022-12-21 09:00:00      93.931960       89.671809  2.276232
2022-12-21 09:15:00     112.193633       97.506378  7.847517
2022-12-21 09:30:00     

                     Power_reality  Power_forecast        APE
TimeStamp                                                    
2022-12-21 10:00:00     136.957567      129.682053   3.887365
2022-12-21 10:15:00     141.614700      130.441132   5.970126
2022-12-21 10:30:00     137.979300      131.613129   3.401496
2022-12-21 10:45:00     147.023000      132.475693   7.772741
2022-12-21 11:00:00     152.526733      134.061691   9.866018
2022-12-21 11:15:00     156.831667      135.049194  11.638547
2022-12-21 11:30:00     157.219633      135.019028  11.861959
2022-12-21 11:45:00     157.189000      136.241302  11.192520
2022-12-21 12:00:00     153.616067      137.740860   8.482249
2022-12-21 12:15:00     146.186433      139.111649   3.780113
2022-12-21 12:30:00     120.401067      140.264664  10.613277
2022-12-21 12:45:00     108.154883      140.429733  17.244708
2022-12-21 13:00:00     133.880433      139.058945   2.766920
2022-12-21 13:15:00     146.340167      135.957382   5.547604
2022-12-

                     Power_reality  Power_forecast        APE
TimeStamp                                                    
2022-12-21 14:00:00     131.570000      127.670731   2.083410
2022-12-21 14:15:00     130.842567      122.373833   4.524912
2022-12-21 14:30:00     103.827590      115.879051   6.439191
2022-12-21 14:45:00      81.654027      109.483833  14.869686
2022-12-21 15:00:00      89.485303      102.953712   7.196278
2022-12-21 15:15:00      92.052640       95.607796   1.899548
2022-12-21 15:30:00      94.972897       87.734016   3.867791
2022-12-21 15:45:00      82.793747       79.040443   2.005420
2022-12-21 16:00:00      69.660900       68.474190   0.634069
2022-12-21 16:15:00      57.992080       58.837086   0.451493
2022-12-21 16:30:00      44.769040       48.140038   1.801151
2022-12-21 16:45:00      38.517430       37.846012   0.358744
2022-12-21 17:00:00      21.051593       28.996887   4.245233
2022-12-21 17:15:00      10.971318       20.480759   5.080970
2022-12-

                     Power_reality  Power_forecast  APE
TimeStamp                                              
2022-12-21 18:00:00            0.0             0.0  0.0
2022-12-21 18:15:00            0.0             0.0  0.0
2022-12-21 18:30:00            0.0             0.0  0.0
2022-12-21 18:45:00            0.0             0.0  0.0
2022-12-21 19:00:00            0.0             0.0  0.0
2022-12-21 19:15:00            0.0             0.0  0.0
2022-12-21 19:30:00            0.0             0.0  0.0
2022-12-21 19:45:00            0.0             0.0  0.0
2022-12-21 20:00:00            0.0             0.0  0.0
2022-12-21 20:15:00            0.0             0.0  0.0
2022-12-21 20:30:00            0.0             0.0  0.0
2022-12-21 20:45:00            0.0             0.0  0.0
2022-12-21 21:00:00            0.0             0.0  0.0
2022-12-21 21:15:00            0.0             0.0  0.0
2022-12-21 21:30:00            0.0             0.0  0.0
2022-12-21 21:45:00            0.0             0

                     Power_reality  Power_forecast        APE
TimeStamp                                                    
2022-12-22 08:30:00      40.379917       56.985271   8.872372
2022-12-22 08:45:00      43.208933       72.954155  15.893107
2022-12-22 09:00:00      49.495120       89.616684  21.437269
2022-12-22 09:15:00      79.405523       99.103073  10.524557
2022-12-22 09:30:00     122.350300      106.073059   8.697059
2022-12-22 09:45:00     143.487600      112.421448  16.598891
2022-12-22 10:00:00     135.612567      117.280609   9.794910
2022-12-22 10:15:00     118.985067      121.336060   1.256154
2022-12-22 10:30:00     121.448500      125.401344   2.112036
2022-12-22 10:45:00     130.761500      128.922424   0.982633
2022-12-22 11:00:00     152.403933      131.815308  11.000666
2022-12-22 11:15:00     158.517633      134.115784  13.038101
2022-12-22 11:30:00     162.481833      136.164230  14.061703
2022-12-22 11:45:00     160.577367      139.039856  11.507662
2022-12-

### Calc MAPE 

In [ ]:
path_files = './test_LN2/transfer_test/add_layer/time_2/'
len_files = os.listdir(path_files)
len(len_files)

In [ ]:
def get_df(name_file):
    phumy_result = pd.read_csv(path_files+name_file)
    phumy_result['TimeStamp'] = pd.to_datetime(phumy_result['TimeStamp'])
    phumy_result = phumy_result.set_index('TimeStamp')    
    start_time = phumy_result.index[0]
    end_time = phumy_result.index[-1]   
    return phumy_result

In [ ]:
path_file_error = './test_LN2/transfer_test/add_layer/time_2/'
error_files = os.listdir(path_file_error)
len(error_files)

In [ ]:
def get_df_error(name_file):
    result_e = pd.read_csv(path_file_error+name_file)
    result_e['TimeStamp'] = pd.to_datetime(result_e['TimeStamp'])
    result_e = result_e.set_index('TimeStamp')    
    start_time = result_e.index[0]
    end_time = result_e.index[-1]   
    return result_e

In [ ]:
time_mape=[]
mean_ape = []
for mape_name_f in error_files:
    result_e = get_df_error(mape_name_f)
    result_e=result_e.between_time(START_TIME_DAY, END_TIME_DAY)
    time_mape.append(result_e.index[0])
    mean_ape.append(result_e['APE'].mean())
mape_df = pd.DataFrame({
    'ATS_MAPE': mean_ape
},index=time_mape
)

In [ ]:
from datetime import datetime
from datetime import date
from datetime import time

In [ ]:
mape_df = mape_df.reset_index()

In [ ]:
mape_df.to_csv('./test_LN2/transfer_test/MAPE_change_layer_2.csv', index = False)

In [ ]:
mape_df = pd.read_csv('./test_LN2/transfer_test/MAPE_change_layer_2.csv')
mape_df['time'] = pd.to_datetime(mape_df['time'])
mape_df = mape_df.set_index('time')

In [ ]:
mape_df['ATS_MAPE'].plot(figsize=(15,6), grid=True,marker = 'o')
plt.title('MAPE_Error')
plt.xlabel("Time")
plt.ylabel("%")

In [ ]:
mape_df = mape_df.reset_index()

In [ ]:
data_ln = mape_df.loc[mape_df['time'].dt.date>=date(year=2022,month=12,day=1)]
data_ln = data_ln.loc[data_ln['time'].dt.date<=date(year=2022,month=12,day=31)]

In [ ]:
data_ln = data_ln.set_index('time')
data_ln['ATS_MAPE'].plot(figsize=(15,6), grid=True,marker = 'o')
plt.title('MAPE_Error')
plt.xlabel("Time")
plt.ylabel("%")

In [ ]:
data_ln[data_ln['ATS_MAPE'] > 15].count()